In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

#### 사용하던 MME df에 LBM column 추가

In [2]:
MME = pd.read_csv('MME_preprocessed_eGFR.csv', encoding = 'euc-kr')

In [3]:
MME

,기수,NIHID,BODYFAT,WEIGHT,HEIGHT,WAIST,TCHL_ORI,AGE,CREATININ_TR,SBP_AVG,eGFR,eGFR_60,SMOKE_2.0,SMOKE_3.0,SEX_1
0,A01,NIH1604000095,13400.0,48.0,157.5,63.0,200.0,43.0,0.90,96.0,78.564168,0.0,0,0,0
1,A01,NIH1604000171,15700.0,75.0,175.5,81.3,215.0,42.0,1.00,95.0,92.420166,0.0,0,1,1
2,A01,NIH1604000362,14700.0,53.6,155.6,76.4,213.0,48.0,0.70,102.0,102.784214,0.0,0,0,0
3,A01,NIH1604000424,19500.0,88.0,174.5,94.5,128.0,47.0,0.70,131.0,112.380813,0.0,0,1,1
4,A01,NIH1604000472,23400.0,68.8,154.7,104.0,200.0,68.0,1.20,120.0,46.548404,1.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19603,A07,NIH1604999446,27300.0,75.4,162.0,87.4,189.0,55.0,0.98,122.0,65.148191,0.0,0,0,0
19604,A07,NIH1604999640,12900.0,56.6,159.1,77.0,178.0,55.0,0.90,97.0,72.213018,0.0,0,0,0
19605,A07,NIH1604999643,17900.0,60.9,164.1,74.0,247.0,55.0,0.88,92.0,74.201919,0.0,0,0,0
19606,A07,NIH1604999732,10500.0,50.8,163.2,81.3,122.0,75.0,1.06,131.0,68.312024,0.0,1,0,1


In [4]:
MME['WEIGHT'] = MME['WEIGHT'] * 1000
MME

,기수,NIHID,BODYFAT,WEIGHT,HEIGHT,WAIST,TCHL_ORI,AGE,CREATININ_TR,SBP_AVG,eGFR,eGFR_60,SMOKE_2.0,SMOKE_3.0,SEX_1
0,A01,NIH1604000095,13400.0,48000.0,157.5,63.0,200.0,43.0,0.90,96.0,78.564168,0.0,0,0,0
1,A01,NIH1604000171,15700.0,75000.0,175.5,81.3,215.0,42.0,1.00,95.0,92.420166,0.0,0,1,1
2,A01,NIH1604000362,14700.0,53600.0,155.6,76.4,213.0,48.0,0.70,102.0,102.784214,0.0,0,0,0
3,A01,NIH1604000424,19500.0,88000.0,174.5,94.5,128.0,47.0,0.70,131.0,112.380813,0.0,0,1,1
4,A01,NIH1604000472,23400.0,68800.0,154.7,104.0,200.0,68.0,1.20,120.0,46.548404,1.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19603,A07,NIH1604999446,27300.0,75400.0,162.0,87.4,189.0,55.0,0.98,122.0,65.148191,0.0,0,0,0
19604,A07,NIH1604999640,12900.0,56600.0,159.1,77.0,178.0,55.0,0.90,97.0,72.213018,0.0,0,0,0
19605,A07,NIH1604999643,17900.0,60900.0,164.1,74.0,247.0,55.0,0.88,92.0,74.201919,0.0,0,0,0
19606,A07,NIH1604999732,10500.0,50800.0,163.2,81.3,122.0,75.0,1.06,131.0,68.312024,0.0,1,0,1


In [5]:
MME['LBM'] = MME['WEIGHT'] - MME['BODYFAT']
MME

,기수,NIHID,BODYFAT,WEIGHT,HEIGHT,WAIST,TCHL_ORI,AGE,CREATININ_TR,SBP_AVG,eGFR,eGFR_60,SMOKE_2.0,SMOKE_3.0,SEX_1,LBM
0,A01,NIH1604000095,13400.0,48000.0,157.5,63.0,200.0,43.0,0.90,96.0,78.564168,0.0,0,0,0,34600.0
1,A01,NIH1604000171,15700.0,75000.0,175.5,81.3,215.0,42.0,1.00,95.0,92.420166,0.0,0,1,1,59300.0
2,A01,NIH1604000362,14700.0,53600.0,155.6,76.4,213.0,48.0,0.70,102.0,102.784214,0.0,0,0,0,38900.0
3,A01,NIH1604000424,19500.0,88000.0,174.5,94.5,128.0,47.0,0.70,131.0,112.380813,0.0,0,1,1,68500.0
4,A01,NIH1604000472,23400.0,68800.0,154.7,104.0,200.0,68.0,1.20,120.0,46.548404,1.0,0,0,0,45400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19603,A07,NIH1604999446,27300.0,75400.0,162.0,87.4,189.0,55.0,0.98,122.0,65.148191,0.0,0,0,0,48100.0
19604,A07,NIH1604999640,12900.0,56600.0,159.1,77.0,178.0,55.0,0.90,97.0,72.213018,0.0,0,0,0,43700.0
19605,A07,NIH1604999643,17900.0,60900.0,164.1,74.0,247.0,55.0,0.88,92.0,74.201919,0.0,0,0,0,43000.0
19606,A07,NIH1604999732,10500.0,50800.0,163.2,81.3,122.0,75.0,1.06,131.0,68.312024,0.0,1,0,1,40300.0


In [6]:
MME.to_csv('MME_preprocessed_eGFR_LBM.csv', encoding = 'euc-kr', index = False)

--------------------------------

#### 검증센터 final에 LBM column 추가

In [7]:
검증센터 = pd.read_csv('건증센터_preprocessed.csv', encoding = 'euc-kr')
검증센터

,체지방,체중,신장,cholesterol,수진시나이,SBP,흡연여부_1.0,흡연여부_2.0,흡연여부_3.0,성별_F,성별_M
0,16747.6,56.2,148.1,212.0,53.0,139.0,1,0,0,1,0
1,13563.2,69.2,178.4,185.0,32.0,126.0,0,0,1,0,1
2,15368.6,51.4,153.8,176.0,44.0,110.0,1,0,0,1,0
3,22774.5,72.3,161.9,214.0,53.0,156.0,1,0,0,1,0
4,23218.0,61.1,150.0,227.0,56.0,134.0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
57823,13588.4,64.4,173.6,256.0,23.0,127.0,0,0,1,0,1
57824,12480.6,67.1,180.4,171.0,23.0,118.0,1,0,0,0,1
57825,8000.4,45.2,164.9,165.0,21.0,116.0,1,0,0,1,0
57826,8573.8,52.6,163.0,181.0,22.0,135.0,1,0,0,0,1


In [8]:
검증센터['체중'] = 검증센터['체중'] * 1000
검증센터

,체지방,체중,신장,cholesterol,수진시나이,SBP,흡연여부_1.0,흡연여부_2.0,흡연여부_3.0,성별_F,성별_M
0,16747.6,56200.0,148.1,212.0,53.0,139.0,1,0,0,1,0
1,13563.2,69200.0,178.4,185.0,32.0,126.0,0,0,1,0,1
2,15368.6,51400.0,153.8,176.0,44.0,110.0,1,0,0,1,0
3,22774.5,72300.0,161.9,214.0,53.0,156.0,1,0,0,1,0
4,23218.0,61100.0,150.0,227.0,56.0,134.0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
57823,13588.4,64400.0,173.6,256.0,23.0,127.0,0,0,1,0,1
57824,12480.6,67100.0,180.4,171.0,23.0,118.0,1,0,0,0,1
57825,8000.4,45200.0,164.9,165.0,21.0,116.0,1,0,0,1,0
57826,8573.8,52600.0,163.0,181.0,22.0,135.0,1,0,0,0,1


In [9]:
검증센터['LBM'] = 검증센터['체중'] - 검증센터['체지방']
검증센터

,체지방,체중,신장,cholesterol,수진시나이,SBP,흡연여부_1.0,흡연여부_2.0,흡연여부_3.0,성별_F,성별_M,LBM
0,16747.6,56200.0,148.1,212.0,53.0,139.0,1,0,0,1,0,39452.4
1,13563.2,69200.0,178.4,185.0,32.0,126.0,0,0,1,0,1,55636.8
2,15368.6,51400.0,153.8,176.0,44.0,110.0,1,0,0,1,0,36031.4
3,22774.5,72300.0,161.9,214.0,53.0,156.0,1,0,0,1,0,49525.5
4,23218.0,61100.0,150.0,227.0,56.0,134.0,1,0,0,1,0,37882.0
...,...,...,...,...,...,...,...,...,...,...,...,...
57823,13588.4,64400.0,173.6,256.0,23.0,127.0,0,0,1,0,1,50811.6
57824,12480.6,67100.0,180.4,171.0,23.0,118.0,1,0,0,0,1,54619.4
57825,8000.4,45200.0,164.9,165.0,21.0,116.0,1,0,0,1,0,37199.6
57826,8573.8,52600.0,163.0,181.0,22.0,135.0,1,0,0,0,1,44026.2


In [11]:
검증센터.to_csv('검증센터_preprocessed_LBM.csv', encoding = 'euc-kr', index = False)